In [1]:
from policyengine_us import Microsimulation
import pandas as pd

In [2]:
baseline = Microsimulation(
    dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5",
)

In [ ]:
OTHER_DEDS = [
    "charitable_deduction",
    "interest_deduction",
    "medical_expense_deduction",
    "casualty_loss_deduction",
    "misc_deduction"
]

df = pd.DataFrame(baseline.calculate_dataframe(["employment_income", "filing_status", "tax_unit_weight"] + OTHER_DEDS, map_to="tax_unit", period=2026))
df["non_salt_deductions"] = df.loc[:, OTHER_DEDS].sum(axis=1)

In [4]:
# Check total tax units
df.tax_unit_weight.sum() / 1e6

204.083056

In [5]:
def get_average_other_deductions(min_wages, max_wages, filing_status):
    filtered = df[df.employment_income.between(min_wages, max_wages) & (df.filing_status == filing_status)]
    total = (filtered.non_salt_deductions * filtered.tax_unit_weight).sum()
    count = filtered.tax_unit_weight.sum()
    print(f"Total: {total}, Count: {count}, Records: {len(filtered)}")
    return total / count

In [6]:
get_average_other_deductions(225_000, 275_000, "JOINT")

Total: 8496808448.0, Count: 939716.5625, Records: 983


9041.885

In [7]:
get_average_other_deductions(450_000, 550_000, "JOINT")

Total: 33081294848.0, Count: 711078.25, Records: 470


46522.723

In [8]:
get_average_other_deductions(900_000, 1_100_000, "JOINT")

Total: 9938963456.0, Count: 311397.6875, Records: 192


31917.268